# NSW  Sensitive List

## List Identifiers
* Sensitive list: __[dr487](https://lists.ala.org.au/speciesListItem/list/dr487)__ (and [dr18457 in test](https://lists-test.ala.org.au/speciesListItem/list/dr18457))

* Collection:  __[dr487](https://collections.ala.org.au/public/show/dr487)__
### Source Data

*  __[Home Page](https://data.bionet.nsw.gov.au)__
*  __[API/Data Info](https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames)__
*  __[Data Information and file downloads](https://www.environment.nsw.gov.au/research-and-publications/publications-search/bionet-sensitive-species-list)__


**Metadata Description**
Classification codes under the Threatened Species Conservation Act 1995: * Presumed Extinct * Critically Endangered * Endangered * Vulnerable * Endangered Population Learn more about the classifications used by the NSW Office of Environment & Heritage Machine-readable downloads can be obtained by searching NSW BioNet, the website for the Atlas of NSW Wildlife, and selecting: 1. Which species or group? = "all species", 2. Legal status = "Select records that fall under one or more categories" then choose "Threatened in NSW" 3. What area? = "Entire area" 4. Period of records? = "All records"

**Metadata URL**
https://data.bionet.nsw.gov.au


Import required libraries
Retrieve dataset from API
Set Project directory

In [1]:
import pandas as pd
import urllib.request, json
import os
import sys

projectDir = "/Users/oco115/PycharmProjects/authoritative-lists/"
statusDir = projectDir + "source-data/status-codes/"
# projectDir = "/Users/new330/IdeaProjects/authoritative-lists/"
sys.path.append(os.path.abspath(projectDir + "source-code/includes"))
# import list_functions as lf


In [ ]:
# %%script echo skipping # comment this line to download the files and save locally

with urllib.request.urlopen("https://data.bionet.nsw.gov.au/biosvcapp/odata/SpeciesNames") as url:
    data = json.loads(url.read().decode())
sensitivelist = pd.json_normalize(data, record_path =['value'])

## Process Sensitive

Create dictionary for location obfuscation based on Category

In [4]:
dictn = {"Category 3": "1km",
       "Category 2": "10km",
       "Category 1": "WITHHOLD"
}
select = ['Category 3','Category 2','Category 1']

Set location generalisation values based on Category

In [6]:
sensitivelist['generalisation'] = sensitivelist['sensitivityClass']
sensitivelist['generalisation'] = sensitivelist['generalisation'].replace(dictn)

Extract records where speciesID equals taxonID and sensitivityClass is one of Category 3, Category 2 or Category1
Set status and sourceStatus

In [7]:
sensitivelist = sensitivelist[(sensitivelist['speciesID'] == sensitivelist['taxonID']) & sensitivelist['sensitivityClass'].isin(select)]
sensitivelist = sensitivelist[['scientificName', 'family', 'genus', 'vernacularName','generalisation', 'sensitivityClass']]
sensitivelist = sensitivelist.rename(columns={'sensitivityClass': 'category'})
sensitivelist

Write dataframe to CSV file - UTF-8 Encoding

In [12]:
print("Writing to CSV")
sensitivelist.to_csv(projectDir + "current-lists/sensitive-lists/NSW-sensitive-20230614.csv",encoding="UTF-8",index=False)
print('Finished writing NSW Sensitive list')

Writing to CSV
Finished writing NSW Sensitive list
